<a href="https://colab.research.google.com/github/AryaveerGupta/Transfer_learning_and_visualisation_of_CNN/blob/main/classification_without_tranfer_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [40]:
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [41]:
import os
# from google.colab import drive
import tensorflow as tf
from tensorflow import keras
from keras.utils.vis_utils import plot_model
from keras.utils import load_img
import numpy as np
import pandas as pd
from keras.layers import Dense
from tensorflow.keras import layers
from tensorflow.keras.initializers import glorot_normal
from tensorflow.keras.callbacks import ModelCheckpoint
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import itertools

# Display
# from IPython.display import Image, display
import matplotlib.cm as cm
from PIL import Image as PILImage

In [42]:
path_in = '/content/drive/MyDrive/Group_20'
random_seed = 10
class_names = ['brain', 'butterfly', 'ewer', 'helicopter', 'ketch']
normalization_layer = tf.keras.layers.Rescaling(1./255)

## read training data
train_ds = tf.keras.utils.image_dataset_from_directory(
    path_in + '/train/',
    batch_size=32,
    seed = random_seed,
    image_size=(224, 224)
).map(lambda x, y: (normalization_layer(x), y))

Found 250 files belonging to 5 classes.


In [43]:
valid_ds = tf.keras.utils.image_dataset_from_directory(
    path_in + '/val/',
    batch_size=32,
    seed = random_seed,
    image_size=(224, 224),
    shuffle=False
).map(lambda x, y: (normalization_layer(x), y))

Found 50 files belonging to 5 classes.


In [44]:
# load test dataset
test_ds = tf.keras.utils.image_dataset_from_directory(
    path_in + '/test/',
    batch_size=32,
    seed = random_seed,
    image_size=(224, 224)
).map(lambda x, y: (normalization_layer(x), y))

Found 100 files belonging to 5 classes.


In [45]:
import keras
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

# Create a Sequential model
model = Sequential()

# Add a 2D convolutional layer with 32 filters, a 3x3 kernel, and 'relu' activation
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(224,224,3)))

# Add a max pooling layer with a 2x2 pool size
model.add(MaxPooling2D(pool_size=(2, 2)))

# Add another 2D convolutional layer with 64 filters, a 3x3 kernel, and 'relu' activation
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))

# Add another max pooling layer with a 2x2 pool size
model.add(MaxPooling2D(pool_size=(2, 2)))

# Flatten the tensor output from the previous layer
model.add(Flatten())

# Add a fully connected layer with 128 units and 'relu' activation
model.add(Dense(128, activation='relu'))

# Add the output layer with the appropriate number of classes and 'softmax' activation
model.add(Dense(5, activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Print a summary of the model architecture
model.summary()


Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_6 (Conv2D)           (None, 222, 222, 32)      896       
                                                                 
 max_pooling2d_6 (MaxPooling  (None, 111, 111, 32)     0         
 2D)                                                             
                                                                 
 conv2d_7 (Conv2D)           (None, 109, 109, 64)      18496     
                                                                 
 max_pooling2d_7 (MaxPooling  (None, 54, 54, 64)       0         
 2D)                                                             
                                                                 
 flatten_3 (Flatten)         (None, 186624)            0         
                                                                 
 dense_6 (Dense)             (None, 128)              

In [46]:

# Define the function to convert labels
def convert_labels(image, label):
    num_classes = 5
    converted_label = tf.one_hot(label, num_classes)
    return image, converted_label

# Apply the label conversion using map function
train_ds = train_ds.map(convert_labels)
valid_ds = valid_ds.map(convert_labels)
test_ds = test_ds.map(convert_labels)




In [47]:
# Train the model
model.fit(train_ds, batch_size=32, epochs=10, validation_data= valid_ds)

Epoch 1/10
8/8 [==============================] - 31s 4s/step - loss: 4.8867 - accuracy: 0.1880 - val_loss: 1.4862 - val_accuracy: 0.2600
Epoch 2/10
8/8 [==============================] - 28s 3s/step - loss: 1.3708 - accuracy: 0.3640 - val_loss: 1.2681 - val_accuracy: 0.5000
Epoch 3/10
8/8 [==============================] - 27s 3s/step - loss: 0.8574 - accuracy: 0.7600 - val_loss: 0.6528 - val_accuracy: 0.8200
Epoch 4/10
8/8 [==============================] - 26s 3s/step - loss: 0.3821 - accuracy: 0.8840 - val_loss: 0.5220 - val_accuracy: 0.8400
Epoch 5/10
8/8 [==============================] - 26s 3s/step - loss: 0.1689 - accuracy: 0.9560 - val_loss: 1.0183 - val_accuracy: 0.7600
Epoch 6/10
8/8 [==============================] - 26s 3s/step - loss: 0.1322 - accuracy: 0.9520 - val_loss: 0.5250 - val_accuracy: 0.8200
Epoch 7/10
8/8 [==============================] - 27s 3s/step - loss: 0.0569 - accuracy: 0.9960 - val_loss: 0.6027 - val_accuracy: 0.8200
Epoch 8/10
8/8 [==================

In [48]:
test_loss ,  test_acc =  model.evaluate(test_ds,verbose = 0 )

In [49]:
print("Test accuracy : ", test_acc)

Test accuracy :  0.6600000262260437
